# Multiclass SVM 구현

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y) # 꽃의 종류(label)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [39]:
y.unique()# 총 세개의 라벨이 존재함


array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
# 데이터 구조 파악하기
iris.head()
# x = sepal_length, sepal_width, petal_length, petal_width
# y = species

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
# 피처와 라벨, 트레인과 테스트 데이터로 나눈다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [7]:
# 표준화 함수
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train) # train data: fit_transform
    test = scaler.transform(test) # test data: transform
    return train, test

X_train, X_test = standardization(X_train, X_test) # Feature에 해당하는 데이터를 표준화해준다.

In [8]:
X_train # -1에서 1 사이의 값으로 표준화되었다.

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

# Multi Class SVM
## 1. One v.s. One
    - n개의 클래스가 있을 때 모든 클래스에 대해 binary 분류를 하고, 가장 많이 승리한 것에 대해 투표하여 결정한다
    - n개의 클래스에 대해 서로 classifier를 가져야 하기 때문에 총 n(n-1)/2개의 classifier가 필요하다
    
### iris의 예
    - y값이 ['setosa' 'versicolor' 'virginica'] 로 세개이므로, 3*2/2 = 3개의 classifier가 필요하다. 
     1) setosa인지 versicolor인지 구분해주는 머신
     2) versicolor인지 virginica인지 구분해주는 머신
     3) virginica인지 setosa인지 구분해주는 머신.
    - 만약 새로운 데이터를 1), 2), 3) 머신에 돌린 결과 각각의 머신이 setosa, virginica, setosa 라고 답했다면, 해당 데이터는 setosa라고 판단한다.

## One v.s. Rest
    - n개의 클래스가 있으면 모든 클래스에 대해 1:n-1로 binary 분류하여 해당 클래스인지 여부를 판단한다.(투표로 결정)
    - 모든 클래스는 classier가 필요하므로 n개의 classifier가 필요하다.
    
### iris의 예
    1) sentosa인지 여부를 구분해주는 머신
    2) versicolor인지 여부를 구분해주는 머신
    3) virginica인지 여부를 구분해주는 머신
    
    - 만약 새로운 데이터를 1), 2), 3) 머신에 돌린 결과 각각 setosa이고, virginica가 아니고, virginica가 아니라고 답변했다면, 새로 들어온 데이터는 sentosa라고 판단한다.

### label을 one-hot 인코딩

In [15]:
print(y_train)

110     virginica
69     versicolor
148     virginica
39         setosa
53     versicolor
          ...    
64     versicolor
91     versicolor
81     versicolor
51     versicolor
0          setosa
Name: species, Length: 120, dtype: object


In [18]:
y_train_onehot = pd.get_dummies(y_train)
print(y_train_onehot)

     setosa  versicolor  virginica
110       0           0          1
69        0           1          0
148       0           0          1
39        1           0          0
53        0           1          0
..      ...         ...        ...
64        0           1          0
91        0           1          0
81        0           1          0
51        0           1          0
0         1           0          0

[120 rows x 3 columns]


### classifier 생성

In [20]:
SVM1 = SVC(kernel ='rbf', C=5, gamma=5)
SVM2 = SVC(kernel ='rbf', C=5, gamma=5)
SVM3 = SVC(kernel ='rbf', C=5, gamma=5)

### 앞서 one-hot 인코딩한 라벨 각각의 binary clasifier 학습

In [23]:
SVM1.fit(X_train,y_train_onehot.iloc[:,0]) # X_train 데이터가 sentosa 인지
SVM2.fit(X_train,y_train_onehot.iloc[:,1]) # X_train 데이터가 versicolor 인지
SVM3.fit(X_train,y_train_onehot.iloc[:,2]) # X_train 데이터가 virginica 인지

SVC(C=5, gamma=5)

In [36]:
print(SVM1.predict(X_test))

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]


In [25]:
print(SVM1.decision_function(X_test))

[-1.12359969 -0.86782512 -0.65599247 -0.50194294 -0.76541147 -0.8819188
  1.07735938 -0.99156769  0.50201986 -0.9984315  -0.84532712  0.17062549
  0.34917127 -0.9813287  -0.72783399 -0.93313988  1.28153212 -0.56827872
 -0.73092732 -0.99670034  0.43553308 -0.96967771 -0.83939495 -1.03305682
 -0.75566609  1.13888006  0.42965012 -1.04268452 -0.93608147 -1.06090982]


## Multiclass SVM - OVM

In [46]:
def OVR_svm(classifier, x, label):
    
    # 동점의 투표결과를 예방하기위해 각각의 클래스에 따른 decision function값을 저장할 distance를 만들 것이다.
    distance = None
    
    for c in classifier:
        if distance is None:
            distance = c.decision_function(x) # decision_fuction을 이용하여 각각의 클래스와 Hyperplane과의 거리를 구한다.
        else:
            distance = np.vstack((distance, c.decision_function(x)))
        
    distance = distance.T
    print("distance: ",distance) 
        
    prediction = [] # 함수의 리턴값
        
    for d in distance:
        prediction.append(labels[d.argmax()]) 
            # 각 라벨에 대한 예측 결과에 대해서 가장 큰 값을 저장
            
    print("prediction :",prediction)
    return prediction

models = [SVM1,SVM2,SVM3]
labels = ['setosa' ,'versicolor' ,'virginica']

prediction = OVR_svm(models, X_test, labels)


distance:  [[-1.12359969  1.37946585 -1.27457139]
 [-0.86782512  0.64158717 -0.78526984]
 [-0.65599247  0.08334644 -0.4283019 ]
 [-0.50194294 -0.37103802 -0.13015479]
 [-0.76541147 -0.2487255   0.02100991]
 [-0.8819188  -0.81469931  0.60666785]
 [ 1.07735938 -1.03237441 -1.04453951]
 [-0.99156769 -0.865664    0.87392972]
 [ 0.50201986 -0.79146099 -0.71179178]
 [-0.9984315   0.98627485 -0.98399934]
 [-0.84532712 -0.74374355  0.54318106]
 [ 0.17062549 -0.65274563 -0.51951719]
 [ 0.34917127 -0.72754177 -0.623009  ]
 [-0.9813287  -0.82140043  0.79694612]
 [-0.72783399  0.49349433 -0.76730253]
 [-0.93313988  0.71459247 -0.77830886]
 [ 1.28153212 -1.11781529 -1.16309745]
 [-0.56827872 -0.16565136 -0.26885348]
 [-0.73092732 -0.22738953 -0.0392679 ]
 [-0.99670034 -0.99528903  0.9921313 ]
 [ 0.43553308 -0.76367034 -0.67308697]
 [-0.96967771 -0.98134121  0.91631483]
 [-0.83939495  0.10485724 -0.28552792]
 [-1.03305682  1.07435138 -1.03901081]
 [-0.75566609 -0.5638643   0.31617703]
 [ 1.13888006 

In [43]:
accuracy_score(y_test,prediction)

0.8666666666666667